In [2]:
import numpy as np
from scipy.ndimage import gaussian_filter

In [3]:
def extract_neighborhood_info(foreground_image, background_image, alpha_for_pixels, x, y, start_dim, sigma_gaussian):
    """
    Retrieves foreground and background neighborhood pixels and weights, along with initial alpha,
    for a given pixel in the unknown area.
    """
    # Image dimensions
    rows, cols, _ = foreground_image.shape
    
    # Half window size
    half_start_dim = (start_dim - 1) // 2
    
    # Index of four vertices of the square neighborhood
    min_x = max(0, x - half_start_dim)
    max_x = min(rows, x + half_start_dim + 1)
    min_y = max(0, y - half_start_dim)
    max_y = min(cols, y + half_start_dim + 1)
    
    # Extract alpha, foreground_image, background_image, neighborhood
    fg_neigh = foreground_image[min_x:max_x, min_y:max_y, :]
    bg_neigh = background_image[min_x:max_x, min_y:max_y, :]
    alpha_neigh = alpha_for_pixels[min_x:max_x, min_y:max_y, :]
    
    # Gaussian distribution area
    init_gaussian = gaussian_filter(np.ones((start_dim, start_dim)), sigma=sigma_gaussian)
    rel_gaussian = init_gaussian[(x - min_x):(x - min_x + start_dim), (y - min_y):(y - min_y + start_dim)]
    rel_gaussian = np.stack([rel_gaussian] * 3, axis=-1)
    
    # Foreground weight = (alpha^2) * gaussian_fall_off
    fg_w = alpha_neigh ** 2 * rel_gaussian
    
    # Background weight = ((1 - alpha)^2) * gaussian_fall_off
    bg_w = (1 - alpha_neigh) ** 2 * rel_gaussian
    
    # Calculate the mean of all the alpha values in the neighborhood as the initial alpha
    init_alpha = np.nanmean(alpha_neigh)
    
    # Assign initial alpha as 0 if it's NaN
    if np.isnan(init_alpha):
        init_alpha = 0
    
    return fg_neigh, bg_neigh, fg_w, bg_w, init_alpha